### Tips for development vs tutorial hygiene:
---
- Keep a scratch notebook (e.g., `prtecan_devel.ipynb`) for experiments.
- Avoid `os.chdir`; use Path objects relative to repository root as in this notebook.
- When a feature stabilizes, port minimal, clear examples into the main tutorial and keep heavy testing in `tests/`.

## Setup

In [ ]:
# Magic commands for development
%load_ext autoreload
%autoreload 2

from pathlib import Path

import arviz as az
import matplotlib.pyplot as plt
import numpy as np

from clophfit import prtecan
from clophfit.fitting.bayes import (
    fit_binding_pymc,
    fit_binding_pymc2,
    fit_binding_pymc_compare,
)
from clophfit.fitting.core import (
    fit_binding_glob,
    fit_binding_glob_recursive,
    fit_binding_glob_recursive_outlier,
    fit_binding_glob_reweighted,
    outlier2,
)
from clophfit.fitting.odr import (
    fit_binding_odr,
    fit_binding_odr_recursive,
    fit_binding_odr_recursive_outlier,
)

# Configure notebook
%matplotlib inline
plt.style.use("seaborn-v0_8")

data_root = Path("tests/Tecan")
l0_dir = data_root / "140220"
l1_dir = data_root / "L1"
l2_dir = data_root / "L2"
l4_dir = data_root / "L4"

In [ ]:
def tit(folder, bg_mth="meansd"):
    tit = prtecan.Titration.fromlistfile(folder / "list.pH.csv", is_ph=1)
    tit.load_additions(folder / "additions.pH")
    tit.load_scheme(folder / "scheme.txt")
    tit.params.bg_mth = bg_mth
    tit.params.bg_adj = True
    return tit


tit = tit(l2_dir)
tit.bg_err

## Discard detection

test:

- E10
- F10
- G09


In [ ]:
plt.plot([np.nanmean(tit.data[1][k] / tit.data[2][k].mean()) for k in tit.fit_keys])

print(
    [
        (t[0], t[1])
        for t in [
            (k, np.nanmean(tit.data[1][k] / tit.data[2][k].mean()))
            for k in tit.fit_keys
        ]
        if t[1] > 2 or t[1] < 1
    ]
)

In [ ]:
## Fitting

In [ ]:
k = "B01"

ds = tit._create_global_ds(k)
ds["y1"].y_err.mean(), ds["y2"].y_err.mean()
ds

In [ ]:
r1 = fit_binding_glob(ds)
r2 = fit_binding_glob(ds, robust=True)
r3 = fit_binding_glob_reweighted(ds, k, threshold=2.5)
r4 = fit_binding_glob_recursive(ds, tol=0.001, max_iterations=100)
r5 = fit_binding_glob_recursive_outlier(ds, tol=0.001, threshold=2)
r6 = outlier2(ds, k, threshold=3, plot_z_scores=True)
r7 = outlier2(ds, k, threshold=3, plot_z_scores=True, error_model="shot-noise")

r8 = fit_binding_odr(r1)
r9 = fit_binding_odr_recursive(r1, tol=0.001, max_iterations=100)
r10 = fit_binding_odr_recursive_outlier(r1, tol=0.001, threshold=3)

n_sd = 0.15 / r7.result.params["K"].stderr
print(n_sd)
r11 = fit_binding_pymc(r7, n_sd=max(n_sd, 1), n_xerr=0.682, ye_scaling=10)
r12 = fit_binding_pymc2(r7, n_sd=max(n_sd, 1), n_xerr=0.682)

buffer_sd = {"y1": r7.dataset["y1"].y_err.mean(), "y2": r7.dataset["y2"].y_err.mean()}
print(buffer_sd)
trace_compare = fit_binding_pymc_compare(
    r7, buffer_sd=buffer_sd, learn_separate_y_mag=True, n_sd=max(n_sd, 1), n_xerr=0.682
)

In [ ]:
ds["y1"].y_err, ds["y2"].y_err

In [ ]:
r7.dataset["y2"].y_err

In [ ]:
r11.figure

In [ ]:
az.summary(r12.mini)

In [ ]:
ds2 = tit._create_ds(k, 2)
outlier2(ds2, error_model="shot-noise").figure

In [ ]:
{"y1": tit.bg_err[1].mean(), "y2": tit.bg_err[2].mean()}

In [ ]:
# You can pass the traces directly to az.compare
comparison_results = az.compare({"single_y_mag": r11.mini, "separate_y_mag": r12.mini})

# The result is a pandas DataFrame.
# The best model has the lowest 'loo' or 'waic' value.
# The 'd_loo' column shows the difference from the best model.
comparison_results